In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-18 05:00:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.75MB/s    in 0.2s    

2021-08-18 05:00:25 (4.75 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url_list ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/final_income_data.csv"
spark.sparkContext.addFile(url_list)
income_data_df = spark.read.csv(SparkFiles.get("final_income_data.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# Show DataFrame
income_data_df.show()

+--------------------+----+-------+
|               state|date| income|
+--------------------+----+-------+
|       United States|2019|68703.0|
|             Alabama|2019|56200.0|
|              Alaska|2019|78394.0|
|             Arizona|2019|70674.0|
|            Arkansas|2019|54539.0|
|          California|2019|78105.0|
|            Colorado|2019|72499.0|
|         Connecticut|2019|87291.0|
|            Delaware|2019|74194.0|
|District of Columbia|2019|93111.0|
|             Florida|2019|58368.0|
|             Georgia|2019|56628.0|
|              Hawaii|2019|88006.0|
|               Idaho|2019|65988.0|
|            Illinois|2019|74399.0|
|             Indiana|2019|66693.0|
|                Iowa|2019|66054.0|
|              Kansas|2019|73151.0|
|            Kentucky|2019|55662.0|
|           Louisiana|2019|51707.0|
+--------------------+----+-------+
only showing top 20 rows



In [6]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ucb-data-group-project-zillow.csaw135fqqkl.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [7]:
# Write DataFrame to income table in RDS
income_data_df.write.jdbc(url=jdbc_url, table='income', mode=mode, properties=config)